In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# download SF crime data
download('https://drive.google.com/open?id=1GVjpKTcXq7FTuJLLWMI8NA_NZBeB47_Z', '~/Downloads')
crime_file = 'SF_Police_Department_Incidents.csv'

In [3]:
# create DataFrame with crime_file
crime_df = pd.read_csv(crime_file)
crime_df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,170513602,WARRANTS,WARRANT ARREST,Friday,06/23/2017,16:00,TARAVAL,"ARREST, BOOKED",2500 Block of 44TH AV,-122.502210,37.739058,"(37.739058349527, -122.502210350969)",17051360263010
1,170817802,OTHER OFFENSES,TRAFFIC VIOLATION,Saturday,10/07/2017,00:57,MISSION,"ARREST, BOOKED",20TH ST / CAPP ST,-122.417964,37.758697,"(37.758696816649, -122.41796448376)",17081780265015
2,170820275,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Saturday,10/07/2017,22:24,NORTHERN,NONE,300 Block of GROVE ST,-122.422348,37.777862,"(37.77786177597, -122.422348143857)",17082027564085
3,130596143,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Saturday,07/20/2013,00:52,MISSION,NONE,500 Block of VALENCIA ST,-122.421739,37.764097,"(37.764097222902, -122.421738508476)",13059614328150
4,130634072,FRAUD,"CREDIT CARD, THEFT OF",Thursday,07/18/2013,17:45,CENTRAL,"ARREST, BOOKED",600 Block of POST ST,-122.413056,37.787645,"(37.787645311178, -122.413056449036)",13063407209310


In [16]:
crime_categories = crime_df['Category'].value_counts()
crime_categories

LARCENY/THEFT                  460588
OTHER OFFENSES                 301689
NON-CRIMINAL                   230964
ASSAULT                        188617
VEHICLE THEFT                  124616
DRUG/NARCOTIC                  117760
VANDALISM                      112247
WARRANTS                        99126
BURGLARY                        88662
SUSPICIOUS OCC                  78101
MISSING PERSON                  63116
ROBBERY                         54425
FRAUD                           40419
SECONDARY CODES                 24983
FORGERY/COUNTERFEITING          22779
WEAPON LAWS                     21402
TRESPASS                        18791
PROSTITUTION                    16608
STOLEN PROPERTY                 11541
SEX OFFENSES, FORCIBLE          11274
DISORDERLY CONDUCT               9916
DRUNKENNESS                      9699
RECOVERED VEHICLE                8645
DRIVING UNDER THE INFLUENCE      5547
KIDNAPPING                       5254
RUNAWAY                          4330
LIQUOR LAWS 

In [5]:
# convert date to year
def date_conversion(date_string):
    
    converted = datetime.strptime(date_string, '%m/%d/%Y').date()
    converted = converted.year
    return converted

crime_df['Date'] = crime_df['Date'].map(lambda x: date_conversion(x))
crime_df
crime_df.head()    

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,170513602,WARRANTS,WARRANT ARREST,Friday,2017,16:00,TARAVAL,"ARREST, BOOKED",2500 Block of 44TH AV,-122.502210,37.739058,"(37.739058349527, -122.502210350969)",17051360263010
1,170817802,OTHER OFFENSES,TRAFFIC VIOLATION,Saturday,2017,00:57,MISSION,"ARREST, BOOKED",20TH ST / CAPP ST,-122.417964,37.758697,"(37.758696816649, -122.41796448376)",17081780265015
2,170820275,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Saturday,2017,22:24,NORTHERN,NONE,300 Block of GROVE ST,-122.422348,37.777862,"(37.77786177597, -122.422348143857)",17082027564085
3,130596143,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Saturday,2013,00:52,MISSION,NONE,500 Block of VALENCIA ST,-122.421739,37.764097,"(37.764097222902, -122.421738508476)",13059614328150
4,130634072,FRAUD,"CREDIT CARD, THEFT OF",Thursday,2013,17:45,CENTRAL,"ARREST, BOOKED",600 Block of POST ST,-122.413056,37.787645,"(37.787645311178, -122.413056449036)",13063407209310


In [95]:
# select categories to analyze
analysis_categories = crime_df.loc[(crime_df['Category']=='VEHICLE THEFT') |
                                   (crime_df['Category']=='LARCENY/THEFT') |
                                   (crime_df['Category']=='ASSAULT') |
                                   (crime_df['Category']=='DRUG/NARCOTIC'), ['Category', 'Date', 'IncidntNum']]
print(f'{len(analysis_categories)} Incidents')

analysis_categories.head()

891581 Incidents


,Category,Date,IncidntNum
10,VEHICLE THEFT,2013,130633096
11,VEHICLE THEFT,2017,170824380
12,LARCENY/THEFT,2011,110273684
15,ASSAULT,2014,140995955
22,LARCENY/THEFT,2014,140815187


In [94]:
# retrive annual total by year
annual_total = analysis_categories.groupby('Date')
annual_total = annual_total['IncidntNum'].count()

annual_total_dict = {
    'Total_Incidents': annual_total,
    'Percent_Change': annual_total.pct_change()
}

annual_total_df = pd.DataFrame(annual_total_dict)
annual_total_df.to_csv('annual_crime_totals.csv')

annual_total_df

,Percent_Change,Total_Incidents
Date,,
2003,NaN,65096
2004,0.001367,65185
2005,-0.023594,63647
2006,-0.117617,56161
2007,-0.015188,55308
2008,0.015929,56189
2009,-0.021125,55002
2010,-0.083961,50384
2011,-0.009983,49881


In [100]:
grouped_categories = analysis_categories.groupby(['Date', 'Category'])
grouped_categories = grouped_categories['IncidntNum'].count()
grouped_categories = grouped_categories.reset_index()
grouped_categories = grouped_categories.pivot(index='Date', columns='Category', values='IncidntNum')

grouped_categories.to_csv('total_crimes_by_category.csv')

grouped_categories

Category,ASSAULT,DRUG/NARCOTIC,LARCENY/THEFT,VEHICLE THEFT
Date,,,,
2003,13461,9917,26393,15325
2004,12899,9897,24505,17884
2005,11601,8533,25319,18194
2006,12449,9069,27352,7291
2007,12518,10560,25770,6460
2008,12681,11648,25807,6053
2009,12284,11950,25585,5183
2010,12387,9205,24446,4346
2011,12279,6935,25905,4762
